## Structural Equation of an Instrumental DAG

given known values of $P_{ABC}(a,b,c)$ for all binary permutations of $A,B,C$:  
$P_{ABC}(a,b,c) =  \sum P_\lambda (\lambda) P_{A|\lambda}(a, \lambda) P_{B|A}(b,a) P_{C|B \lambda}(c,b,\lambda) \qquad \forall \lambda \in [0,card]$  
we find set of values of $P_\lambda (\lambda), \space P_{A|\lambda}(a, \lambda), \space P_{C|B \lambda}(c,b,\lambda)$ that match this constraints, we then find the set of three values that maximize $P_{C |do(B)}(C,B)$


note: can change the cardinality of the hidden variable lamda.

In [1]:
import gurobipy as gp
from gurobipy import GRB
import itertools
import numpy as np

In [2]:
card_A = 2
card_B = 2
card_C = 2
card = 4
instrumental = False


observable_probs = np.arange(card_A * card_B * card_C)
observable_probs = observable_probs / observable_probs.sum()
observable_probs = observable_probs.reshape(card_A, card_B, card_C)

dist = {}
# defining a p distribution based on cardinality of A, B, C
for a, b, c in itertools.product(range(card_A), range(card_B), range(card_C)):
    prob = observable_probs[a, b, c]
    dist[(a, b, c)] = prob

print(dist)

{(0, 0, 0): 0.0, (0, 0, 1): 0.03571428571428571, (0, 1, 0): 0.07142857142857142, (0, 1, 1): 0.10714285714285714, (1, 0, 0): 0.14285714285714285, (1, 0, 1): 0.17857142857142858, (1, 1, 0): 0.21428571428571427, (1, 1, 1): 0.25}


In [3]:
e = 1/100
#e = 0
dist_XYZ = {
    (0,0,0): (0.3- e) * 0.5,
    (1,0,0): (0.7- e) * 0.5,
    (0,0,1): (0.7- e) * 0.5,
    (1,1,1): (0.3- e) * 0.5,

    (0,1,1): e * 0.5,
    (1,0,1): e * 0.5,
    (0,1,0): e * 0.5,
    (1,1,0): e * 0.5

}

dist_ABC = {(z, x, y): v for (x,y,z),v in dist_XYZ.items()}
dist=dist_ABC

distribution feasible?

In [4]:
sum(dist.values()) == 1

True

$P_{B|A}(B,A)$

In [5]:
def P_B_given_A(B,A):
    P_AB = sum([dist[(a,b,c)] for a,b,c in dist if a==A and b==B])
    P_A = sum([dist[(a,b,c)] for a,b,c in dist if a==A])

    return P_AB/P_A

$P_Z(Z)$

In [6]:
# P_A(a) = sum_{B,C} P_ABC(a,b,c)
def P_A(a):
    return sum([dist[(a, B, C)] for B in range(card_B) for C in range(card_C)])

$P_\lambda (\lambda), \space P_{A|\lambda} (A, \lambda), \space P_{C|B,\lambda} (C, B, \lambda)$

In [7]:
m = gp.Model()
#m.Params.LogToConsole = 0
print("Instrumental: ", instrumental)

# variables (MVars)
P_l = m.addMVar(shape = card, vtype=GRB.CONTINUOUS, name="P_l", lb=0, ub=1)
P_C_given_B_l = m.addMVar(shape = (card_C, card_B, card), vtype=GRB.CONTINUOUS, name="P_C_given_B_l", lb=0, ub=1)

# first case:
P_A_given_l = m.addMVar(shape = (card_A, card), vtype=GRB.CONTINUOUS, name="P_A_given_l", lb=0, ub=1) # [(a1, l1), (a2, l2), ...]
# instrumental DAG:
P_B_given_A_l = m.addMVar(shape = (card_B, card_A, card), vtype=GRB.CONTINUOUS, name="P_A_given_l", lb=0, ub=1)


## can't add products of three variables, so we add a new helper variable
tripple_prod = m.addMVar(shape=(card_C, card_B, card, card), vtype=GRB.CONTINUOUS, name="c do b lamdbda time p_lambda", lb=0, ub=1) 
p_C_do_B = m.addMVar(shape=(card_C, card_B), vtype=GRB.CONTINUOUS, name="c do b", lb=0, ub=1)
m.update()

for a, b, c in itertools.product(range(card_A), range(card_B), range(card_C)):
    P_ABC = dist[(a, b, c)]
    RHS_obs = gp.LinExpr()
    RHS_do = gp.LinExpr()    
    for l in range(card):
        m.addConstr(tripple_prod[b, c, l] == P_l[l] * P_C_given_B_l[c, b, l])
        ##############
        if instrumental:
            RHS_obs += tripple_prod[b, c, l]*P_B_given_A_l[b, a, l]*P_A(a) # instrumental DAG
        else:
            RHS_obs += tripple_prod[b, c, l]*P_B_given_A(b,a)*P_A_given_l[a, l] # first case
        ##############
        RHS_do += tripple_prod[b, c, l]
    # probability distribution to symbolic equations equality constraint 
    m.addConstr(P_ABC == RHS_obs)
    m.addConstr(p_C_do_B[c, b] == RHS_do)


for l in range(card):
    m.addConstr(gp.quicksum(P_l[l] for l in range(card)) == 1, "sum_P_l = 1")
    m.addConstr(gp.quicksum(P_A_given_l[a, l] for a in [0, 1]) == 1, f"sum_P_A_given_l_{l} = 1")
    
    # only in card_B = 2
    m.addConstr(gp.quicksum(P_C_given_B_l[c, 0, l] for c in [0, 1]) == 1, f"sum_P_C_given_B_l_0_{l} = 1")
    m.addConstr(gp.quicksum(P_C_given_B_l[c, 1, l] for c in [0, 1]) == 1, f"sum_P_C_given_B_l_1_{l} = 1")

    m.addConstr(gp.quicksum(P_B_given_A_l[b, 0, l] for b in [0, 1]) == 1, f"sum_P_C_given_B_l_0_{l} = 1")
    m.addConstr(gp.quicksum(P_B_given_A_l[b, 1, l] for b in [0, 1]) == 1, f"sum_P_C_given_B_l_1_{l} = 1")

Set parameter Username
Academic license - for non-commercial use only - expires 2025-01-09


Instrumental:  False


In [8]:
# can take (range(card_C), range(card_B), "max" or else is min) as input
def main(c,b):
    print(f"optimizing P(C={c}|do(B={b}))...")
    m.setObjective(p_C_do_B[c,b], GRB.MINIMIZE)
    m.optimize()
    min_val = p_C_do_B[c,b].X.item()

    m.setObjective(p_C_do_B[c,b], GRB.MAXIMIZE)
    m.optimize()
    max_val = p_C_do_B[c,b].X.item()

    print("\nmin value: ", min_val)
    print("max value: ", max_val)
    print("distance:", max_val - min_val)

$\text{max/min }(P_{C|\text{do}(B)})$ over the three unknowns.

In [9]:
main(1,1)

optimizing P(C=1|do(B=1))...
Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (win64 - Windows 11.0 (22621.2))

CPU model: 11th Gen Intel(R) Core(TM) i7-11800H @ 2.30GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Optimize a model with 56 rows, 112 columns and 216 nonzeros
Model fingerprint: 0x7bf53aef
Model has 160 quadratic constraints
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  QMatrix range    [3e-01, 1e+00]
  QLMatrix range   [1e+00, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
  QRHS range       [5e-03, 3e-01]
Presolve removed 27 rows and 16 columns

Continuous model is non-convex -- solving as a MIP

Presolve removed 39 rows and 68 columns
Presolve time: 0.02s
Presolved: 217 rows, 76 columns, 460 nonzeros
Presolved model has 48 bilinear constraint(s)
Variable types: 76 continuous, 0 integer (0 binary)

Root relaxation: objecti